<a href="https://colab.research.google.com/github/zhangxs131/Text_Classification/blob/main/tf_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#tf-idf +贝叶斯

文本分类，数据集 THUCNEW title 新闻标题分类


##下载数据

In [2]:
!pip install datasets

     |████████████████████████████████| 342 kB 5.0 MB/s 
     |████████████████████████████████| 136 kB 55.8 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 1.1 MB 55.9 MB/s 
     |████████████████████████████████| 212 kB 62.0 MB/s 
     |████████████████████████████████| 127 kB 59.9 MB/s 
     |████████████████████████████████| 144 kB 50.5 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 271 kB 76.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [22]:
#下载停用词表
!wget https://raw.githubusercontent.com/goto456/stopwords/master/cn_stopwords.txt

--2022-05-11 12:41:49--  https://raw.githubusercontent.com/goto456/stopwords/master/cn_stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4717 (4.6K) [text/plain]
Saving to: ‘cn_stopwords.txt’

cn_stopwords.txt    100%[===================>]   4.61K  --.-KB/s    in 0s      

2022-05-11 12:41:49 (55.1 MB/s) - ‘cn_stopwords.txt’ saved [4717/4717]



In [3]:
from datasets import load_dataset

cnews=load_dataset('seamew/THUCNewsTitle')
print(cnews['train'][0])
cnews


Using custom data configuration default


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset thuc_news_title downloaded and prepared to /root/.cache/huggingface/datasets/seamew___thuc_news_title/default/0.0.0/b3df30999854cbe65ae45110e895b2fa88c14975f2185c1f43d9b7ca85b5f679. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'text': '星际争霸2免费月卡派送，获奖名单', 'label': 4}


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
})

In [20]:
cnews['train'].features['label']

ClassLabel(num_classes=10, names=['education', 'entertainment', 'fashion', 'finance', 'game', 'politic', 'society', 'sport', 'stock', 'technology'], id=None)

In [21]:
cnews['train'].to_csv('train.csv',index=False)
cnews['validation'].to_csv('validation.csv',index=False)
cnews['test'].to_csv('test.csv',index=False)

Creating CSV from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

558555

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB
from sklearn.metrics import f1_score,accuracy_score,recall_score,precision_score,confusion_matrix
import pandas as pd
import jieba
import warnings



warnings.filterwarnings('ignore')

train_data=pd.read_csv('train.csv')
valid_data=pd.read_csv('validation.csv')

#打开停用词表
with open('cn_stopwords.txt','r',encoding='utf-8') as f:
    stopwords=f.readlines()



#tf-idf
tf_idf=TfidfVectorizer(
    tokenizer=jieba.lcut,
    max_features=1000,
    stop_words=stopwords,
)
train=tf_idf.fit_transform(train_data['text'])
valid=tf_idf.transform(valid_data['text'])


#bayie分类
bayes=MultinomialNB()
bayes.fit(train,train_data['label'])



#评估
predict=bayes.predict(valid)

acc=accuracy_score(predict,valid_data['label'])
re=recall_score(predict,valid_data['label'],average='weighted')
f1=f1_score(predict,valid_data['label'],average='weighted')
p=precision_score(predict,valid_data['label'],average='weighted')


print('tf-idf特征+bayes分类结果为acc:{:.3f},recall:{:.3f},f1_score{:.3f},precision_score:{:.3f}'.format(acc,re,f1,p))



tf-idf特征+bayes分类结果为acc:0.822,recall:0.822,f1_score0.822,precision_score:0.824


In [39]:
#tf
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB
from sklearn.metrics import f1_score,accuracy_score,recall_score,precision_score,confusion_matrix
import pandas as pd
import jieba
import warnings



warnings.filterwarnings('ignore')

train_data=pd.read_csv('train.csv')
valid_data=pd.read_csv('validation.csv')

#打开停用词表
with open('cn_stopwords.txt','r',encoding='utf-8') as f:
    stopwords=f.readlines()



#tf
tf=CountVectorizer(
    tokenizer=jieba.lcut,
    max_features=1000,
    stop_words=stopwords,
)
train=tf.fit_transform(train_data['text'])
valid=tf.transform(valid_data['text'])


#bayie分类
bayes=MultinomialNB()
bayes.fit(train,train_data['label'])



#评估
predict=bayes.predict(valid)

acc=accuracy_score(predict,valid_data['label'])
re=recall_score(predict,valid_data['label'],average='weighted')
f1=f1_score(predict,valid_data['label'],average='weighted')
p=precision_score(predict,valid_data['label'],average='weighted')


print('tf特征+bayes分类结果为acc:{:.3f},recall:{:.3f},f1_score{:.3f},precision_score:{:.3f}'.format(acc,re,f1,p))

tf特征+bayes分类结果为acc:0.826,recall:0.826,f1_score0.826,precision_score:0.828


In [43]:
#进行一下数据清洗,去掉非汉字的词语
import re

def clean_text(text):
  result=[]
  for i in text:
    result.append(re.sub('[^\u4e00-\u9fa5]+', ' ', i))
  return result


In [44]:
#tf
tf=CountVectorizer(
    tokenizer=jieba.lcut,
    max_features=1000,
    stop_words=stopwords,
)
train=tf.fit_transform(clean_text(train_data['text']))
valid=tf.transform(clean_text(valid_data['text']))


#bayie分类
bayes=MultinomialNB()
bayes.fit(train,train_data['label'])



#评估
predict=bayes.predict(valid)

acc=accuracy_score(predict,valid_data['label'])
re=recall_score(predict,valid_data['label'],average='weighted')
f1=f1_score(predict,valid_data['label'],average='weighted')
p=precision_score(predict,valid_data['label'],average='weighted')


print('数据清洗后，tf特征+bayes分类结果为acc:{:.3f},recall:{:.3f},f1_score{:.3f},precision_score:{:.3f}'.format(acc,re,f1,p))

数据清洗后，tf特征+bayes分类结果为acc:0.806,recall:0.806,f1_score0.805,precision_score:0.808


还有点降低，好吧，看来非中文字符有的还有些用，那么在看看不加停用词表呢

In [45]:
#tf
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB
from sklearn.metrics import f1_score,accuracy_score,recall_score,precision_score,confusion_matrix
import pandas as pd
import jieba
import warnings



warnings.filterwarnings('ignore')

train_data=pd.read_csv('train.csv')
valid_data=pd.read_csv('validation.csv')


#tf
tf=CountVectorizer(
    tokenizer=jieba.lcut,
    max_features=1000,
)
train=tf.fit_transform(train_data['text'])
valid=tf.transform(valid_data['text'])


#bayie分类
bayes=MultinomialNB()
bayes.fit(train,train_data['label'])



#评估
predict=bayes.predict(valid)

acc=accuracy_score(predict,valid_data['label'])
re=recall_score(predict,valid_data['label'],average='weighted')
f1=f1_score(predict,valid_data['label'],average='weighted')
p=precision_score(predict,valid_data['label'],average='weighted')


print('未添加停用词表，tf特征+bayes分类结果为acc:{:.3f},recall:{:.3f},f1_score{:.3f},precision_score:{:.3f}'.format(acc,re,f1,p))

未添加停用词表，tf特征+bayes分类结果为acc:0.826,recall:0.826,f1_score0.826,precision_score:0.828


感觉根本没变化，可能几乎没停用词，下面改变下tf特征数，50000个句子，特征词个数，挑了一会发现20000时候已经可以准确率92.5，再高就几乎没啥变化了。

In [57]:
#tf
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB
from sklearn.metrics import f1_score,accuracy_score,recall_score,precision_score,confusion_matrix
import pandas as pd
import jieba
import warnings



warnings.filterwarnings('ignore')

train_data=pd.read_csv('train.csv')
valid_data=pd.read_csv('validation.csv')


#tf
tf=CountVectorizer(
    tokenizer=jieba.lcut,
    max_features=20000,
)
train=tf.fit_transform(train_data['text'])
valid=tf.transform(valid_data['text'])


#bayie分类
bayes=MultinomialNB()
bayes.fit(train,train_data['label'])



#评估
predict=bayes.predict(valid)


acc=accuracy_score(predict,valid_data['label'])
re=recall_score(predict,valid_data['label'],average='weighted')
f1=f1_score(predict,valid_data['label'],average='weighted')
p=precision_score(predict,valid_data['label'],average='weighted')


print('20000特征数，tf特征+bayes分类结果为acc:{:.3f},recall:{:.3f},f1_score{:.3f},precision_score:{:.3f}'.format(acc,re,f1,p))

20000特征数，tf特征+bayes分类结果为acc:0.925,recall:0.925,f1_score0.925,precision_score:0.925


我觉得数据应该是多项式分布，我再看看高斯分布效果咋样需要把稀疏矩阵转为密集矩阵，20000维直接崩了，特征少了效果也一般

#总结一下

感觉这机器学习方法也可以呀，稍微弄弄就10分类短文本分类就达到了92.5准确率。
训练集50000，验证集5000。